# Feature Engineering – Transaction History

This section focuses on processing the **transaction_history** table to create features suitable for **customer churn prediction**. The goal is to capture customer behavior in terms of **monetary value, purchase frequency, recency, and account tenure**.

## Features Created from Transaction History

- **customer_id**: Unique identifier for each customer.
- **total_spent**: Total amount spent by the customer across all transactions.
- **avg_spent**: Average amount spent per transaction, reflecting spending consistency.
- **spending_frequency**: Number of transactions per unit of account age, showing purchase regularity.
- **days_since_last_purchase**: Number of days since the customer's most recent transaction, a key indicator of potential churn.
- **account_age**: Total duration the customer has been active, providing insight into loyalty and tenure.

### Notes

These features combine **RFM analysis (Recency, Frequency, Monetary)** with **tenure information**, forming a solid foundation for **churn prediction modeling**. This table will be joined with other customer data tables for a comprehensive churn analysis.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from narwhals.stable.v1 import Datetime

In [6]:
transactions = pd.read_csv('Datasets/csv_files/crm_transaction_history.csv')
transactions.head(100)

,customer_id,transaction_id,transaction_date,amount_spent,product_category
0,1,7194,2022-03-27,416.50,Electronics
1,2,7250,2022-08-08,54.96,Clothing
2,2,9660,2022-07-25,197.50,Electronics
3,2,2998,2022-01-25,101.31,Furniture
4,2,1228,2022-07-24,397.37,Clothing
...,...,...,...,...,...
95,18,1171,2022-08-30,97.23,Furniture
96,18,6170,2022-04-28,281.73,Books
97,18,1028,2022-02-08,154.09,Furniture
98,18,9033,2022-01-14,254.22,Books


In [8]:
transactions['product_category'].unique()

array(['Electronics', 'Clothing', 'Furniture', 'Groceries', 'Books'],
      dtype=object)

In [9]:
from itertools import count

transactions_new = transactions.groupby('customer_id', as_index = False).agg(total_amount_spent = ('amount_spent',"sum"),
                                                                             number_of_transactions = ('transaction_id',"count"),
                                                                             last_purchase = ('transaction_date',"max"),
                                                                             first_purchase = ('transaction_date',"min"),
                                                                             no_electronics = ('product_category',(lambda x: (x == 'Electronics').sum())),
                                                                             no_furniture = ('product_category',(lambda x: (x == 'Furniture').sum())),
                                                                             no_clothing = ('product_category',(lambda x: (x == 'Clothing').sum())),
                                                                             no_groceries = ('product_category',(lambda x: (x == 'Groceries').sum())),
                                                                             no_books = ('product_category',(lambda x: (x == 'Books').sum())))

transactions_new['last_purchase'] = pd.to_datetime(transactions_new['last_purchase'])
transactions_new['first_purchase'] = pd.to_datetime(transactions_new['first_purchase'])
today = pd.Timestamp.today().normalize()
transactions_new['days_since_last_purchase'] = (today - transactions_new['last_purchase']).dt.days
transactions_new['average_spent'] = (transactions_new['total_amount_spent'] / transactions_new['number_of_transactions'])
transactions_new['account_age'] = (transactions_new['last_purchase'] - transactions_new['first_purchase']).dt.days
transactions_new.drop(['last_purchase','first_purchase'], axis=1, inplace=True)
transactions_new.head()

,customer_id,total_amount_spent,number_of_transactions,no_electronics,no_furniture,no_clothing,no_groceries,no_books,days_since_last_purchase,average_spent,account_age
0,1,416.50,1,1,0,0,0,0,1278,416.50000,0
1,2,1547.42,7,3,1,2,1,0,1041,221.06000,314
2,3,1702.98,6,0,2,1,2,1,1083,283.83000,239
3,4,917.29,5,2,1,1,1,0,1003,183.45800,219
4,5,2001.49,8,3,2,0,3,0,1009,250.18625,303


In [10]:
transactions_new['spending_frequnecy'] = np.where(transactions_new['account_age'] > 0,transactions_new['number_of_transactions']/transactions_new['account_age'],
0)
transactions_new.head(100)


,customer_id,total_amount_spent,number_of_transactions,no_electronics,no_furniture,no_clothing,no_groceries,no_books,days_since_last_purchase,average_spent,account_age,spending_frequnecy
0,1,416.50,1,1,0,0,0,0,1278,416.50000,0,0.000000
1,2,1547.42,7,3,1,2,1,0,1041,221.06000,314,0.022293
2,3,1702.98,6,0,2,1,2,1,1083,283.83000,239,0.025105
3,4,917.29,5,2,1,1,1,0,1003,183.45800,219,0.022831
4,5,2001.49,8,3,2,0,3,0,1009,250.18625,303,0.026403
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,161.84,1,1,0,0,0,0,1128,161.84000,0,0.000000
96,97,553.65,2,0,2,0,0,0,1110,276.82500,138,0.014493
97,98,893.76,5,2,0,1,1,1,1049,178.75200,301,0.016611
98,99,889.75,5,1,0,1,2,1,1051,177.95000,288,0.017361


In [11]:
transactions_new.to_csv('transactions_history', index=False)